In [79]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, numpy.random as rand, cv2, os

external_path = '/Volumes/LaCie/20230310/Generated_Data'
importance_path = './230514_importance_results'

for path_obj in os.scandir(importance_path):
   if not os.path.isdir(path_obj.path):
      continue

   if not os.path.isdir(f'{path_obj.path}/ben_plots'):
      os.mkdir(f'{path_obj.path}/ben_plots')

   # Grab the results "info.csv" file for noise and name purposes
   meta = pd.read_csv(f'{path_obj.path}/info.csv')
   # Don't think this sort is necessary, did it just in case
   meta.sort_values(by='lgt', inplace=True)
   noise_mag = meta.loc[0, 'noise_mag']
   file = meta.loc[0, 'file']
   og_data = pd.read_pickle(f'{external_path}/{file}')
   noise = noise_mag*rand.randn(len(og_data))
   og_data['A'] += noise
   # Plot OG data with all types of importance files provided
   for sub in os.scandir(path_obj):
      if not sub.name.startswith('imp'):
         continue
      data = og_data.copy()
      importances = pd.read_pickle(sub.path)
      glbl = []
      for v in data['v'].unique():
         t = data.loc[data['v'] == v]
         t.reset_index(inplace=True, drop=True)
         n = len(t)
         # Current data in OG data extractio
         fwd = t.loc[:n//2-1,'A'].tolist()
         rev = t.loc[n//2:,'A'].tolist()[::-1]
         srs = [v]*len(fwd)
         glbl.append(fwd)
         glbl.append(rev)
         glbl.append(srs)
      # Shape of global (18, n), n = length of voltage window
      arr = np.array(glbl)
      # Shape after rescaling (18, 1000) --> (width, height) --> (col, row)
      rescaled = cv2.resize(arr, (1000, 18))
      df = pd.DataFrame(rescaled)
      
      # Just to get the color bar
      minv, maxv = -1, 1
      cmap = plt.cm.coolwarm
      norm = plt.Normalize(minv, maxv)
      sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
      sm.set_array([])

      fig, ax = plt.subplots()
      for i in range(6):
         # Only want first and last scan rate for now, lazy approach
         if 0 < i < 5:
            continue
         start = i*3
         end = start + 3
         # Blocks of 3 rows, row 1 is forward current, row 2 is reverse, row 3 is ignored
         data = df.iloc[start:end, :]
         data.reset_index(inplace=True, drop=True)
         fwd = data.iloc[0,:].tolist()
         # Grab importance data for forward currents of appropriate scan rate
         fwd_imp = importances.iloc[start,:]
         fwd_opacity = abs(fwd_imp)
         rev = data.iloc[1,:].tolist()
         # Same as above, reverse currents
         rev_imp = importances.iloc[start+1,:]
         rev_opacity = abs(rev_imp)
         n = range(len(fwd))
         ax.scatter(n, fwd, c=fwd_imp, cmap='seismic', s=10, alpha=fwd_opacity)
         ax.scatter(n, rev, c=rev_imp, cmap='seismic', s=10, alpha=rev_opacity)
      ax.set_title(sub.name)
      fig.colorbar(sm, ax=ax, label='Values')
      fig.savefig(f'{path_obj.path}/ben_plots/{sub.name}', dpi=300)
      plt.close(fig)
importances 

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-0.048333,-0.042672,-0.051261,-0.043261,-0.022456,-0.020388,-0.009705,0.000757,0.004615,0.006067,...,-0.051345,-0.027164,0.031182,0.068381,0.019632,0.031955,0.055196,0.054603,0.015542,0.028608
1,-0.018319,-0.021738,-0.031854,-0.030804,-0.010085,-0.010664,-0.009192,-0.004666,-0.000439,0.003307,...,0.098931,0.132879,0.096379,0.083360,0.125232,0.122559,0.108676,0.095517,0.028412,0.059442
2,-0.030434,-0.028044,-0.042281,-0.048597,-0.023382,-0.024038,-0.015413,-0.002611,0.005563,0.010048,...,0.157916,0.146319,0.109720,0.102925,0.079324,0.098467,0.111486,0.127177,0.078897,0.086510
3,-0.046841,-0.047911,-0.064323,-0.060059,-0.046933,-0.039631,-0.001183,-0.008375,0.000136,0.013717,...,-0.026928,-0.027915,0.058527,0.079189,0.083239,0.083706,0.057146,0.067041,0.106144,0.101641
4,-0.035282,-0.035008,-0.055967,-0.069068,-0.049123,-0.052894,-0.041898,-0.030038,-0.023374,-0.013396,...,-0.145500,-0.040614,-0.023644,0.001147,0.090573,0.073015,0.057242,0.059385,-0.021566,0.006862
5,-0.031430,-0.055169,-0.095044,-0.109452,-0.088973,-0.072051,-0.048818,-0.025225,-0.008354,-0.009696,...,0.134748,0.084802,0.051930,0.010293,0.021928,-0.005021,-0.082011,-0.043735,-0.018683,0.043849
6,0.014169,0.001246,-0.003038,-0.008657,-0.015084,-0.015928,-0.017672,-0.017055,-0.023582,-0.030546,...,0.147034,0.185685,0.192976,0.162920,0.111372,0.116554,0.142653,0.140477,0.057266,0.022961
7,0.006817,0.009296,0.000108,-0.018258,-0.008220,-0.015192,-0.023353,-0.029703,-0.038365,-0.034619,...,0.076029,0.074153,0.042434,0.029867,0.074174,0.110759,0.073156,0.071780,0.060208,0.023063
8,0.017092,0.009029,-0.000870,-0.008800,-0.003481,-0.012607,-0.007118,-0.019275,-0.016095,-0.022220,...,0.051631,0.106878,0.063461,0.069948,0.100173,0.103005,0.136624,0.113103,0.059652,0.034683
9,0.035346,0.034533,0.030220,0.004213,0.002875,-0.014258,-0.021889,-0.025267,-0.036254,-0.041052,...,0.187095,0.251547,0.266586,0.287939,0.250422,0.200834,0.155303,0.170715,0.071082,0.079133
